## Read streaming data from Kinesis
### Set-up

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
#ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access #key ID').collect()[0]['Access key ID']
#SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secret key
#ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# new version
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


### Capture pin data stream


In [0]:
df_pin_stream = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0ea9a6e05a33-pin') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

pin_schema = StructType([
StructField("index", StringType()),
StructField("unique_id", StringType()),
StructField("title", StringType()),
StructField("description", StringType()),
StructField("follower_count", StringType()),
StructField("poster_name", StringType()),
StructField("tag_list", StringType()),
StructField("is_image_or_video", StringType()),
StructField("image_src", StringType()),
StructField("downloaded", StringType()),
StructField("save_location", StringType()),
StructField("category", StringType())
])

df_pin = df_pin_stream \
.selectExpr("CAST(data as STRING)") \
.select(from_json(col("data"), pin_schema).alias("pin_data")) \
.select("pin_data.*")


#### clean pin data

In [0]:
df_pin_cleaned = df_pin.withColumn("category", when(df_pin["category"].isNull(), None).otherwise(df_pin["category"]))
df_pin_cleaned = df_pin.withColumn("description", when(df_pin["description"].isNull(), None).otherwise(df_pin["description"]))
df_pin_cleaned = df_pin_cleaned.withColumn(
    "follower_count",
    when(col("follower_count").endswith("k"), 
         regexp_replace(col("follower_count"), "k", "").cast("int") * 1000)
    .when(col("follower_count").endswith("M"), 
          regexp_replace(col("follower_count"), "M", "").cast("int") * 1000000)
    .otherwise(col("follower_count").cast("int"))
)
df_pin_cleaned = df_pin_cleaned.withColumn(
    "follower_count",
    when(
        df_pin_cleaned["follower_count"].cast("int").isNotNull(),
        df_pin_cleaned["follower_count"].cast("int")
    ).otherwise(None)
)
df_pin_cleaned.withColumn("downloaded",col("downloaded").cast("int"))
df_pin_cleaned = df_pin_cleaned.withColumn("save_location",regexp_replace(col("save_location"), "^Local save in ", ""))
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")
desired_column_order = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin_cleaned = df_pin_cleaned.select(desired_column_order)
df_pin_cleaned.withColumn("ind",col("ind").cast("int"))

df_pin_cleaned.writeStream \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
.table("0ea9a6e05a33_pin_table")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2998023767729582> in <module> 
 23 df_pin_cleaned . withColumn ( "ind" , col ( "ind" ) . cast ( "int" ) ) 
 24 
 ---> 25 df_pin_cleaned . writeStream \ 
 26 . format ( "delta" ) \ 
 27 . outputMode ( "append" ) \ 

 /databricks/spark/python/pyspark/sql/streaming.py in table (self, tableName) 
 1231 """Alias for the toTable API
 1232 """
 -> 1233 return self . toTable ( tableName ) 
 1234 
 1235 def toTable(self, tableName, format=None, outputMode=None, partitionBy=None, queryName=None,

 /databricks/spark/python/pyspark/sql/streaming.py in toTable (self, tableName, format, outputMode, partitionBy, queryName, **options) 
 1301 if queryName is not None : 
 1302 self . queryName ( queryName ) 
 -> 1303 return self . _sq ( self . _jwrite . toTable ( tableName ) ) 
 1304 
 1305 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o5180.toTable.
: java.util.concurrent.TimeoutException: Stream Execution thread for stream [id = 0339b0ac-0973-403e-bebc-d3826ee7c8ef, runId = ea501f88-7abe-4ad7-9d3a-74373139b4d9] failed to stop within 15000 milliseconds (specified by spark.sql.streaming.stopTimeout). See the cause on what was being executed in the streaming query thread.
	at org.apache.spark.sql.execution.streaming.StreamExecution.interruptAndAwaitExecutionThreadTermination(StreamExecution.scala:560)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.stop(MicroBatchExecution.scala:216)
	at org.apache.spark.sql.streaming.StreamingQueryManager.$anonfun$startQuery$7(StreamingQueryManager.scala:416)
	at org.apache.spark.sql.streaming.StreamingQueryManager.$anonfun$startQuery$7$adapted(StreamingQueryManager.scala:414)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.streaming.StreamingQueryManager.startQuery(StreamingQueryManager.scala:414)
	at org.apache.spark.sql.streaming.DataStreamWriter.startQuery(DataStreamWriter.scala:459)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:438)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:254)
	at org.apache.spark.sql.streaming.DataStreamWriter.writeToV1Table$1(DataStreamWriter.scala:348)
	at org.apache.spark.sql.streaming.DataStreamWriter.toTable(DataStreamWriter.scala:361)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
C

### Capture geo data stream


In [0]:
df_geo_stream = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0ea9a6e05a33-geo') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

geo_schema = StructType([
StructField("ind", StringType()),
StructField("timestamp", StringType()),
StructField("latitude", DoubleType()),
StructField("longitude", DoubleType()),
StructField("country", StringType())
])

df_geo = df_geo_stream \
.selectExpr("CAST(data as STRING)") \
.select(from_json(col("data"), geo_schema).alias("geo_data")) \
.select("geo_data.*")    

#### clean geo data

In [0]:
df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))
df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp(col("timestamp")))
df_geo_cleaned = df_geo_cleaned.select("ind", "country", "coordinates", "timestamp")

df_geo_cleaned.writeStream \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
.table("0ea9a6e05a33_geo_table")

Out[32]: <pyspark.sql.streaming.StreamingQuery at 0x7fabb1a281f0>

### Capture user data


In [0]:
df_user_streams = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0ea9a6e05a33-user') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

user_schema = StructType([
    StructField("ind", StringType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("age", IntegerType()),
    StructField("date_joined", TimestampType())
])

df_user = df_user_streams \
    .selectExpr("CAST(data as STRING)") \
    .select(from_json(col("data"), user_schema).alias("user_data")) \
    .select("user_data.*")

#### Clean user data

In [0]:
df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))
df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")
df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp(col("date_joined"), "yyyy-MM-dd'T'HH:mm:ss"))
df_user_cleaned = df_user_cleaned.select("ind", "user_name", "age", "date_joined")
     
df_user_cleaned.writeStream \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
.table("0ea9a6e05a33_user_table")

Out[35]: <pyspark.sql.streaming.StreamingQuery at 0x7fabb1a384c0>